In [1]:
# Libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
images_gray = np.load('./l/gray_scale.npy')
images_lab = np.load('./ab/ab/ab1.npy')
#images_lab = np.append(images_lab,np.load('./ab/ab/ab2.npy'), axis=0)
#images_lab = np.append(images_lab,np.load('./ab/ab/ab3.npy'), axis=0)
print(images_lab.shape)
print(images_gray.shape)

(10000, 224, 224, 2)
(25000, 224, 224)


In [2]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
from tensorflow import keras

import os
import cv2 # for image resizing
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)

sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras
print(tf.__version__)

Using TensorFlow backend.


1.10.0


In [40]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import *
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
import keras.backend as K
from matplotlib.pyplot import imsave
from skimage.color import lab2rgb
from keras.applications import InceptionResNetV2
import skimage.color as skc
import skimage.transform as skt

In [4]:
images_gray = np.load('./l/gray_scale.npy')
images_lab = np.load('./ab/ab/ab2.npy')
#images_lab = np.append(images_lab,np.load('./ab/ab/ab2.npy'), axis=0)
#images_lab = np.append(images_lab,np.load('./ab/ab/ab3.npy'), axis=0)
print(images_lab.shape)
print(images_gray.shape)

train_gray = images_gray[10000:14000,:,:]
train_lab = images_lab[:4000,:,:,:]

#val_gray = images_gray[8000:10000:,:]
#val_lab = images_lab[8000:10000,:,:]

train_gray = train_gray.reshape(4000,224,224,1)
#val_gray = val_gray.reshape(2000,224,224,1)
#val_gray.shape
del images_gray
del images_lab

(10000, 224, 224, 2)
(25000, 224, 224)


In [5]:
def build_simple_model():
    input_img = Input(shape=(224,224,1), name="input_img")
    x = Conv2D(filters=64, kernel_size=(3,3), activation="relu",padding="same", name="Conv1_1")(input_img)
    x = Conv2D(filters=64, kernel_size=(3,3), activation="relu",padding="same", name="Conv1_2")(x)
    x = BatchNormalization(name="bn_1")(x)
    x = MaxPooling2D(pool_size=(2,2),name="pool_1")(x)
    x = Dropout(rate=0.2)(x)
    
    x = Conv2D(filters=128,kernel_size=(3,3), activation="relu",padding="same", name="Conv2_1")(x)
    x = Conv2D(filters=128,kernel_size=(3,3), activation="relu",padding="same", name="Conv2_2")(x)
    x = BatchNormalization(name="bn_2")(x)
    x = MaxPooling2D(pool_size=(2,2),name="pool_2")(x)
    x = Dropout(rate=0.2)(x)
    
    x = Conv2D(filters=256, kernel_size=(3,3), activation="relu",padding="same", name="Conv3_1")(x)
    x = Conv2D(filters=256, kernel_size=(3,3), activation="relu",padding="same", name="Conv3_2")(x)
    x = BatchNormalization(name="bn_3")(x)
    x = MaxPooling2D(pool_size=(2,2),name="pool_3")(x)
    x = Dropout(rate=0.2)(x)
    
    x = Conv2D(filters=512, kernel_size=(3,3), activation="relu",padding="same", name="Conv4_1")(x)
    x = Conv2D(filters=256, kernel_size=(3,3), activation="relu",padding="same", name="Conv4_2")(x)
    x = Conv2D(filters=128, kernel_size=(3,3), activation="relu",padding="same", name="Conv4_3")(x)
    x = Dropout(rate=0.2)(x)
    
    x = UpSampling2D((2,2), name="UpSampling5_1")(x)
    x = Conv2D(filters=64, kernel_size=(3,3), activation="relu",padding="same", name="Conv5_1")(x)
    
    x = UpSampling2D((2,2), name="UpSampling6_1")(x)
    x = Conv2D(filters=32, kernel_size=(3,3), activation="relu",padding="same", name="Conv6_1")(x)
    
    x = Conv2D(filters=2, kernel_size=(3,3), activation="tanh",padding="same", name="Conv7_1")(x)
    x = UpSampling2D((2,2), name="UpSampling7_1")(x)
    
    model = Model(input_img, x)
    model.compile(optimizer='rmsprop', loss='mse', metrics=["accuracy",'mse'])
    return model

model = build_simple_model()
model.summary()
    

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_img (InputLayer)       (None, 224, 224, 1)       0         
_________________________________________________________________
Conv1_1 (Conv2D)             (None, 224, 224, 64)      640       
_________________________________________________________________
Conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
bn_1 (BatchNormalization)    (None, 224, 224, 64)      256       
_________________________________________________________________
pool_1 (MaxPooling2D)        (None, 112, 112, 64)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 112, 112, 64)      0         
_________________________________________________________________
Conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
__________

In [6]:
save_model_path = './small_conv_net.h5'
cp = tf.keras.callbacks.ModelCheckpoint(filepath=save_model_path, monitor='mse', verbose=1)
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.8 ** x)

In [7]:
batch_size = 16
num_epochs = 20


In [8]:
# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)
# Generate training data

def image_a_b_gen(Xtrain, Ytrain, batch_size):
    for X_batch, Y_batch in datagen.flow(Xtrain, Ytrain, batch_size=batch_size):
        X_batch = X_batch / 255
        Y_batch = Y_batch / 255
        yield (X_batch, Y_batch)

In [9]:
def val_a_b_gen(Xval, Yval, batch_size):
    for X_batch, Y_batch in datagen.flow(Xval, Yval, batch_size=batch_size):
        X_batch = Xval / 255
        Y_batch = Yval / 255
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

In [84]:
model.fit_generator(image_a_b_gen(train_gray, train_lab, batch_size), steps_per_epoch=250, epochs=num_epochs, callbacks=[cp])

Epoch 1/20
250/250 [==============================] - 120s 482ms/step - loss: 0.0043 - acc: 0.5318 - mean_squared_error: 0.0043

Epoch 00001: saving model to ./small_conv_net.h5
Epoch 2/20
250/250 [==============================] - 120s 479ms/step - loss: 0.0043 - acc: 0.5317 - mean_squared_error: 0.0043

Epoch 00002: saving model to ./small_conv_net.h5
Epoch 3/20
250/250 [==============================] - 121s 484ms/step - loss: 0.0043 - acc: 0.5349 - mean_squared_error: 0.0043

Epoch 00003: saving model to ./small_conv_net.h5
Epoch 4/20
250/250 [==============================] - 121s 485ms/step - loss: 0.0043 - acc: 0.5306 - mean_squared_error: 0.0043

Epoch 00004: saving model to ./small_conv_net.h5
Epoch 5/20
250/250 [==============================] - 122s 486ms/step - loss: 0.0043 - acc: 0.5284 - mean_squared_error: 0.0043

Epoch 00005: saving model to ./small_conv_net.h5
Epoch 6/20
250/250 [==============================] - 121s 482ms/step - loss: 0.0043 - acc: 0.5303 - mean_squa

In [10]:
model.load_weights("./small_conv_net.h5")

In [11]:
images_gray = np.load('./l/gray_scale.npy')
images_lab = np.load('./ab/ab/ab1.npy')
#images_lab = np.append(images_lab,np.load('./ab/ab/ab2.npy'), axis=0)
#images_lab = np.append(images_lab,np.load('./ab/ab/ab3.npy'), axis=0)
print(images_lab.shape)
print(images_gray.shape)
test_gray = images_gray[8000:8200,:,:]
test_lab = images_lab[8000:8200,:,:,:]

#val_gray = images_gray[8000:10000:,:]
#val_lab = images_lab[8000:10000,:,:]

test_gray = test_gray.reshape(200,224,224,1)
test_lab = test_lab / 255
test_gray = test_gray / 255
#val_gray = val_gray.reshape(2000,224,224,1)
#val_gray.shape
del images_gray
del images_lab


(10000, 224, 224, 2)
(25000, 224, 224)


In [12]:
# Evaluation on test dataset
test_loss = model.evaluate(test_gray, test_lab, batch_size=16)
print("Loss on test set: ", test_loss)
#print("Accuracy on test set: ", test_score)

200/200 [==============================] - 10s 51ms/step
Loss on test set:  [0.003736745063215494, 0.5476729917526245, 0.003736745179630816]


In [21]:
# Test model
output = model.predict(test_gray)
test_gray_reshape = test_gray.reshape((200,224,224))
#img_ = np.zeros((200,224,224,3))
# Output colorizations
for i in range(len(output)):
        cur = np.zeros((224, 224, 3))
        cur[:,:,0] = test_gray_reshape[i][:,:] * 255
        cur[:,:,1:] = output[i] * 255
        cur = cur.astype('uint8')
        img_ = cv2.cvtColor(cur, cv2.COLOR_LAB2RGB)
        gray = cv2.cvtColor(img_, cv2.COLOR_RGB2GRAY)
        #test_gray_reshape[i] = cv2.cvtColor(test_gray_reshape[i], cv2.)
        imsave("result/img_"+str(i)+".png", img_)
        imsave("result/gray_"+str(i)+".png",gray)

In [25]:
#Load weights
inception = InceptionResNetV2(weights=None, include_top=True)
inception.load_weights('inception_resnet_v2_weights_tf_dim_ordering_tf_kernels.h5')
inception.graph = tf.get_default_graph()

embed_input = Input(shape=(1000,))


In [88]:
#Encoder
encoder_input = Input(shape=(224, 224, 1,))
encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(encoder_input)
#128
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
#64
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
#32
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)

#Fusion
fusion_output = RepeatVector(28 * 28)(embed_input) 
fusion_output = Reshape(([28, 28, 1000]))(fusion_output)
fusion_output = concatenate([encoder_output, fusion_output], axis=3) 
fusion_output = Conv2D(256, (1, 1), activation='relu', padding='same')(fusion_output) 

#Decoder
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(fusion_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
#64
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
#128
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
#256

In [89]:
model = Model(inputs=[encoder_input, embed_input], outputs=decoder_output)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 224, 224, 1)  0                                            
__________________________________________________________________________________________________
conv2d_442 (Conv2D)             (None, 112, 112, 64) 640         input_7[0][0]                    
__________________________________________________________________________________________________
conv2d_443 (Conv2D)             (None, 112, 112, 128 73856       conv2d_442[0][0]                 
__________________________________________________________________________________________________
conv2d_444 (Conv2D)             (None, 56, 56, 128)  147584      conv2d_443[0][0]                 
__________________________________________________________________________________________________
conv2d_445

In [77]:
#Create embedding
def create_inception_embedding(grayscaled_rgb):
    grayscaled_rgb_resized = []
    for i in grayscaled_rgb:
        i = skt.resize(i, (299, 299, 3), mode='constant')
        grayscaled_rgb_resized.append(i)
    grayscaled_rgb_resized = np.array(grayscaled_rgb_resized)
    with inception.graph.as_default():
        embed = inception.predict(grayscaled_rgb_resized[:,:,:,:,1])
    return embed


In [78]:
# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.4,
        zoom_range=0.4,
        rotation_range=40,
        horizontal_flip=True)

#Generate training data
batch_size = 20

def image_a_b_gen(Xtrain, Ytrain, batch_size):
    for X_batch, Y_batch in datagen.flow(Xtrain, Ytrain, batch_size=batch_size):
        X_batch = X_batch / 255
        Y_batch = Y_batch / 255
        grayscaled_rgb = skc.gray2rgb(X_batch)
        embed = create_inception_embedding(grayscaled_rgb)
        yield ([X_batch, embed], Y_batch)

In [79]:
save_model_path = './small_conv_net.h5'
cp = tf.keras.callbacks.ModelCheckpoint(filepath=save_model_path, monitor='mse', verbose=1)
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.8 ** x)
batch_size = 16
num_epochs = 20

In [80]:
model.compile(optimizer='adam', loss='mse')
model.fit_generator(image_a_b_gen(train_gray, train_lab, batch_size), steps_per_epoch=250, epochs=num_epochs, callbacks=[cp])


Epoch 1/20


C:\Users\debji\Anaconda3\envs\tensorflow\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


ValueError: Error when checking input: expected input_4 to have shape (256, 256, 1) but got array with shape (224, 224, 1)

In [90]:
images_gray = np.load('./l/gray_scale.npy')
images_lab = np.load('./ab/ab/ab2.npy')
#images_lab = np.append(images_lab,np.load('./ab/ab/ab2.npy'), axis=0)
#images_lab = np.append(images_lab,np.load('./ab/ab/ab3.npy'), axis=0)
print(images_lab.shape)
print(images_gray.shape)

train_gray = images_gray[10000:14000,:,:]
train_lab = images_lab[:4000,:,:,:]

#val_gray = images_gray[8000:10000:,:]
#val_lab = images_lab[8000:10000,:,:]

train_gray = train_gray.reshape(4000,224,224,1)

#val_gray = val_gray.reshape(2000,224,224,1)
#val_gray.shape
del images_gray
del images_lab

(10000, 224, 224, 2)
(25000, 224, 224)


In [91]:
model.compile(optimizer='adam', loss='mse')
model.fit_generator(image_a_b_gen(train_gray, train_lab, batch_size), steps_per_epoch=250, epochs=num_epochs, callbacks=[cp])


Epoch 1/20


C:\Users\debji\Anaconda3\envs\tensorflow\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


250/250 [==============================] - 1312s 5s/step - loss: 0.0112

Epoch 00001: saving model to ./small_conv_net.h5
Epoch 2/20
250/250 [==============================] - 577s 2s/step - loss: 0.0050

Epoch 00002: saving model to ./small_conv_net.h5
Epoch 3/20
250/250 [==============================] - 586s 2s/step - loss: 0.0046

Epoch 00003: saving model to ./small_conv_net.h5
Epoch 4/20
250/250 [==============================] - 580s 2s/step - loss: 0.0045

Epoch 00004: saving model to ./small_conv_net.h5
Epoch 5/20
250/250 [==============================] - 599s 2s/step - loss: 0.0044

Epoch 00005: saving model to ./small_conv_net.h5
Epoch 6/20
250/250 [==============================] - 600s 2s/step - loss: 0.0043

Epoch 00006: saving model to ./small_conv_net.h5
Epoch 7/20
250/250 [==============================] - 593s 2s/step - loss: 0.0043

Epoch 00007: saving model to ./small_conv_net.h5
Epoch 8/20
250/250 [==============================] - 588s 2s/step - loss: 0.0042

Epo